In [1]:
import pandas as pd
from statsmodels.tsa.api import VAR

# pobranie danych z Yahoo Finance
df = pd.read_csv("https://query1.finance.yahoo.com/v7/finance/download/EURUSD=X?period1=1483228800&period2=1640995200&interval=1d&events=history&includeAdjustedClose=true")
df.set_index('Date', inplace=True)

# przekształcenie różnicowe
df_diff = df.diff().dropna()

# dopasowanie modelu VAR
model = VAR(df_diff)
results = model.fit(2) # drugi parametr to liczba opóźnień (lag)

# wyświetlenie podsumowania wyników modelu
print(results.summary())

# wykonanie prognozy na kolejne 10 dni
forecast = results.forecast(df_diff.values[-2:], steps=10)
print(forecast)


/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


LinAlgError: 6-th leading minor of the array is not positive definite

In [9]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import rmse, aic

# Pobranie danych z Yahoo Finance
df = yf.download('EURUSD=X', start='2017-01-01', end='2022-02-28', interval='1d')
df.dropna(inplace=True)

# Model VAR z bootstrapingiem
def VAR_bootstrap(data, lags):
    # Liczba iteracji bootstrapingu
    n_bootstraps = 100

    # Inicjalizacja macierzy do przechowywania wyników
    n_variables = len(data.columns)
    coefs = np.zeros((n_bootstraps, lags, n_variables, n_variables))
    rmse_scores = np.zeros((n_bootstraps, n_variables))
    aic_scores = np.zeros(n_bootstraps)

    # Wykonanie bootstrapingu
    for i in range(n_bootstraps):
        # Losowe próbkowanie danych
        boot_data = data.sample(frac=1, replace=True)

        # Dopasowanie modelu VAR
        model = VAR(boot_data)
        results = model.fit(lags)

        # Zapisanie wyników
        coefs[i, :, :, :] = results.coefs
        # rmse_scores[i, :] = rmse(boot_data.iloc[1:, :], results.fittedvalues, axis=0)
        rmse_scores[i, :] = rmse(boot_data.iloc[1:, :], results.fittedvalues, axis=0)
        aic_scores[i] = results.aic

    # Obliczenie średnich i odchyleń standardowych dla wyników
    mean_coefs = np.mean(coefs, axis=0)
    std_coefs = np.std(coefs, axis=0)
    mean_rmse = np.mean(rmse_scores, axis=0)
    std_rmse = np.std(rmse_scores, axis=0)
    mean_aic = np.mean(aic_scores)
    std_aic = np.std(aic_scores)

    # Zwrócenie wyników
    return mean_coefs, std_coefs, mean_rmse, std_rmse, mean_aic, std_aic

# Przekształcenie danych do postaci zmiennych różnicowych
df_diff = df.diff().dropna()

# Dopasowanie modelu VAR z bootstrapingiem dla 2 opóźnień (lag)
mean_coefs, std_coefs, mean_rmse, std_rmse, mean_aic, std_aic = VAR_bootstrap(df_diff, 2)

# Wyświetlenie wyników
print(mean_coefs, std_coefs, mean_rmse, std_rmse, mean_aic, std_aic)


[*********************100%***********************]  1 of 1 completed


/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/kkepins-macwro_1/.local/share/virtualenvs/data-science-upskills-eseJr18D/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: operands could not be broadcast together with shapes (1340,6) (1339,6) 